In [1]:
import numpy as np
import cbp

# 1. Construct nodes

## 1.1 Construct Variable node

In [2]:
rv_dim = 3
varnode_1 = cbp.node.VarNode(rv_dim)
fix_marginal = np.ones(3) / 3
varnode_2 = cbp.node.VarNode(rv_dim,constrained_marginal=fix_marginal)

## 1.2 Construct Factor node

In [3]:
connected_var = ["VarNode_000","VarNode_001"]
factor_node = cbp.node.FactorNode(connected_var, np.ones((3,3)))

# 2. Construct Graph

In [4]:
graph = cbp.graph.GraphModel()
graph.add_varnode(varnode_1)
graph.add_varnode(varnode_2)
graph.add_factornode(factor_node)

'FactorNode_000'

# 3. Run inference

In [5]:
graph.run_bp() # run iterative scaling
# graph.run_cnp() # run cnp

([0.0], 1)

# 4. Access to the marginal

In [6]:
varnode = graph.get_node('VarNode_000') # same node as varnode_1
print(varnode.marginal())
print(factor_node.marginal())

[0.33333333 0.33333333 0.33333333]
[[0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]]


In [7]:
print(graph.export_convergence_marginals())

{'FactorNode_000': array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]]), 'VarNode_000': array([0.33333333, 0.33333333, 0.33333333]), 'VarNode_001': array([0.33333333, 0.33333333, 0.33333333])}
